# SUMO Script for Gathering Traffic Data
- In order to run this, ensure you have SUMO download and configured properly. You will also need XQuartz running in order for the SUMO gui to run

In [1]:
import numpy as np
import os
import sys
import traci

# Ensure SUMO is installed and setup correctly.
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)


In [22]:
# Command to run the simulation with the given config file and save the tripinfo to tripinfo.xml
# NOTE: --scale scales up the traffic in the simulation. 100.0 means 100x the traffic.
traci.start(['sumo-gui', '-c', '../sumo_files/config.sumo.cfg', '--tripinfo-output', '../sumo_files/tripinfo.xml', '--start', '--quit-on-end', '--scale', '1'])

# Vehicle ID to track
vehicle_id = "10"

# Variables to store departure and arrival times
departure_time = 0
arrival_time = 0
time_loss = 0
waiting_time_at_red_lights = {}

# Start simulation explicitly
traci.simulationStep()

# Run the simulation for 1000 steps
step = 0
i = 0
while step < 500:
    # Run the simulation
    traci.simulationStep()
    
    # Check if the vehicle is on the road
    if vehicle_id in traci.vehicle.getIDList():
        # Record departure time if not done already
        if departure_time == 0:
            departure_time = traci.simulation.getTime()
        
        # Track the waiting time at red lights. If the waiting time is 0, the vehicle is not moving
        # If the waiting time is greater than 0, the vehicle is waiting at a red light. Need to track time to get the max value.
        if traci.vehicle.getWaitingTime(vehicle_id) == 0:
            i += 1
        elif traci.vehicle.getWaitingTime(vehicle_id) > 0:
            if i not in waiting_time_at_red_lights:
                waiting_time_at_red_lights[i] = []
            waiting_time_at_red_lights[i].append(traci.vehicle  .getWaitingTime(vehicle_id))
            
        # For some reason, the road ID here is the Junction ID where the vehicle ends at. Really weird.
        # It's defined in the network file and includes the specific edge it's suppose to end on.
        if traci.vehicle.getRoadID(vehicle_id) == ":202799427_4":
            arrival_time = traci.simulation.getTime()
            time_loss = traci.vehicle.getTimeLoss(vehicle_id)

    step += 1

# In the dict, get the max value in the [] for each key and add to get the total waiting time.
total_red_light_waiting_time = 0
for key in waiting_time_at_red_lights:
    total_red_light_waiting_time += max(waiting_time_at_red_lights[key])

# Calculate total travel time
total_travel_time = arrival_time - departure_time

print("- Departure time for vehicle:", departure_time)
print("- Arrival time for vehicle:", arrival_time)
print("- Total time waiting at red lights: ", total_red_light_waiting_time)
print("- Time loss: ", time_loss)
print("- Total travel time for vehicle:", total_travel_time)

traci.close()

 Retrying in 1 seconds
- Departure time for vehicle: 11.0
- Arrival time for vehicle: 107.0
- Total time waiting at red lights:  26.0
- Time loss:  73.22865297059192
- Total travel time for vehicle: 96.0
